1. Prepare the data

In [12]:
# Useful libraries
import numpy as np
import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.datasets import load_diabetes, make_regression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.feature_selection import SelectPercentile, f_regression

np.random.seed(42)


In [13]:
# Read in the data
url = "https://raw.githubusercontent.com/rhodes-byu/stat-486/main/data/OceanicFisheries/ocean_data.csv"
df = pd.read_csv(url)
print(df.head())

# Drop columns that are not needed
df = df.drop(columns=['Wea', 'Cloud_Typ', 'Cloud_Amt', 'Visibility'])
print(df.head())

   T_degC   Salnty  O2ml_L  Depthm  Bottom_D  Wind_Spd  Dry_T  Wet_T  \
0  16.830  33.8510   5.560      65    1337.0      14.0   16.5   15.5   
1   9.262  33.8481   2.729     140    1202.0       5.0   15.0   13.0   
2  15.390  33.4260   5.990      30    3871.0      10.0   18.8   17.6   
3  14.540  32.9470   5.840      42    4018.0      14.0   16.9   16.1   
4   7.410  34.1810   1.000     300    4058.0      21.0   16.3   14.9   

                       Wea      Cloud_Typ                  Cloud_Amt  \
0                      NaN            NaN                        NaN   
1            Partly Cloudy        Stratus  1/10 or less but not zero   
2  Continuous blowing snow        Stratus                      10/10   
3  Continuous blowing snow  Stratocumulus                      10/10   
4            Partly Cloudy  Stratocumulus               7/10 to 8/10   

     Visibility  
0           NaN  
1  10km to 20km  
2  10km to 20km  
3   4km to 10km  
4   4km to 10km  
   T_degC   Salnty  O2ml_L

In [14]:
# Create training data and testing data
X = df.drop(columns=['T_degC'])
y = df['T_degC']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    random_state=307, 
    test_size=0.25
)

print(X_train.shape, X_test.shape)

(5329, 7) (1777, 7)


2. Pipelines

In [15]:
# a) build a pipeline
pipe = Pipeline([
    ('impute', SimpleImputer(strategy='mean')),
    ('poly', PolynomialFeatures(degree=2, include_bias=False)),
    ('scale', StandardScaler()),
    ('model', LinearRegression())
])

# b) fit pipline to training data
pipe.fit(X_train, y_train)

# c) report on the training MSE
y_train_pred = pipe.predict(X_train)
train_mse = mean_squared_error(y_train, y_train_pred)
print(f"Training MSE: {train_mse}")

# d) report on the test MSE
y_test_pred = pipe.predict(X_test)
test_mse = mean_squared_error(y_test, y_test_pred)
print(f"Test MSE: {test_mse}")

# e) how does the test MSE compare to the variance of ytest? What does this say about the predictability of your model?
y_test_variance = np.var(y_test)
print(f"Variance of y_test: {y_test_variance}")
test_mse / y_test_variance


Training MSE: 1.5350956362420822
Test MSE: 1.7543308610948438
Variance of y_test: 14.8162872136925


np.float64(0.11840556515896747)

3. Feature Importance

In [16]:
# a) use pipeline to order features by magnitude of coefficients
betas = pipe.named_steps['model'].coef_
feature_names = pipe.named_steps['poly'].get_feature_names_out(X_train.columns)

# df of coefficients and feature names
coef_df = pd.DataFrame({
    'feature': feature_names,
    'coefficient': betas
})

# order by magnitude of coefficients
coef_df['abs_coef'] = coef_df['coefficient'].abs()
coef_df_sorted = coef_df.sort_values('abs_coef', ascending=False)

coef_df_sorted.head(10)

# b top 3 largest positive
top_positive = coef_df_sorted.sort_values('coefficient', ascending=False).head(3)
top_positive

# c top 3 largest negative
top_negative = coef_df_sorted.sort_values('coefficient').head(3)
top_negative

# d y-intercept
intercept = pipe.named_steps['model'].intercept_
print(f"y-intercept: {intercept}")

y-intercept: 11.607865265528682


4. Hyperparameter Tuning with Grid Search

In [17]:
# a) fit a KNN model using the same pipline
pipe_knn = Pipeline([
  ('impute', SimpleImputer()),
  ('poly', PolynomialFeatures(include_bias=False)),
  ('scale', StandardScaler()),
  ('model', KNeighborsRegressor())
])

# b) use grid search with 10-fold cross validation to find the best hyperparameters
param_grid = {
    'impute__strategy': ['mean', 'median'],
    'poly__degree': [1, 2, 3],
    'model__n_neighbors': list(range(5, 101, 5)),
    'model__weights': ['uniform', 'distance']
}

gs = GridSearchCV(
    pipe_knn,
    param_grid=param_grid,
    scoring='neg_mean_squared_error',
    cv=10
)

# c) report the best hyperparameters by MSE
gs.fit(X_train, y_train)
best_params = gs.best_params_
best_mse = -gs.best_score_

best_params, best_mse


({'impute__strategy': 'median',
  'model__n_neighbors': 10,
  'model__weights': 'distance',
  'poly__degree': 3},
 np.float64(1.1942865460094283))

In [18]:
# e) use optimized pipline to predit on test data
best_model = gs.best_estimator_
y_test_pred_knn = best_model.predict(X_test)
test_mse_knn = mean_squared_error(y_test, y_test_pred_knn)
print(f"KNN Test MSE: {test_mse_knn}")

KNN Test MSE: 1.2398437245052845


5. Randomized Grid Search

In [19]:
# Complete the steps from quesiton 4 but using RandomizedSearchCV

# a) fit a KNN model using the same pipline
pipe_knn = Pipeline([
  ('impute', SimpleImputer()),
  ('poly', PolynomialFeatures(include_bias=False)),
  ('scale', StandardScaler()),
  ('model', KNeighborsRegressor())
])

# b) use grid search with 10-fold cross validation to find the best hyperparameters
param_dist = {
    'impute__strategy': ['mean', 'median'],
    'poly__degree': [1, 2, 3],
    'model__n_neighbors': list(range(5, 101, 5)),
    'model__weights': ['uniform', 'distance']
}

rs = RandomizedSearchCV(
    pipe_knn,
    param_distributions=param_dist,
    scoring='neg_mean_squared_error',
    cv=10
)

# c) report the best hyperparameters by MSE
rs.fit(X_train, y_train)
best_params = rs.best_params_
best_mse = -rs.best_score_

best_params, best_mse

# e) use optimized pipline to predit on test data
best_model_rs = rs.best_estimator_
y_test_pred_knn = best_model_rs.predict(X_test)
test_mse_knn = mean_squared_error(y_test, y_test_pred_knn)
print(f"KNN Test MSE: {test_mse_knn}")


KNN Test MSE: 1.2989312431187976


6. Advanced Pipelines

In [20]:
# Adding back in all columns

df = pd.read_csv(url)
print(df.head())

   T_degC   Salnty  O2ml_L  Depthm  Bottom_D  Wind_Spd  Dry_T  Wet_T  \
0  16.830  33.8510   5.560      65    1337.0      14.0   16.5   15.5   
1   9.262  33.8481   2.729     140    1202.0       5.0   15.0   13.0   
2  15.390  33.4260   5.990      30    3871.0      10.0   18.8   17.6   
3  14.540  32.9470   5.840      42    4018.0      14.0   16.9   16.1   
4   7.410  34.1810   1.000     300    4058.0      21.0   16.3   14.9   

                       Wea      Cloud_Typ                  Cloud_Amt  \
0                      NaN            NaN                        NaN   
1            Partly Cloudy        Stratus  1/10 or less but not zero   
2  Continuous blowing snow        Stratus                      10/10   
3  Continuous blowing snow  Stratocumulus                      10/10   
4            Partly Cloudy  Stratocumulus               7/10 to 8/10   

     Visibility  
0           NaN  
1  10km to 20km  
2  10km to 20km  
3   4km to 10km  
4   4km to 10km  


In [21]:
# Create training data and testing data
X = df.drop(columns=['T_degC'])
y = df['T_degC']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    random_state=307, 
    test_size=0.25
)

print(X_train.shape, X_test.shape)

numeric_cols = X.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = X.select_dtypes(include=['object', 'category']).columns

num_pipeline = Pipeline(steps=[
    ('num_impute', SimpleImputer(strategy='mean')),
    ('poly', PolynomialFeatures(include_bias=False)),
    ('scale', StandardScaler())
])

cat_pipeline = Pipeline(steps=[
    ('cat_impute', SimpleImputer(strategy='most_frequent')),
    ('dummy', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(transformers=[
    ('num', num_pipeline, numeric_cols),
    ('cat', cat_pipeline, categorical_cols)
])

pipe = Pipeline(steps=[
    ('preprocess', preprocessor),
    ('dimension', SelectPercentile(f_regression, percentile=50)),
    ('model', LinearRegression())
])

# a) fit pipeline to training data
pipe.fit(X_train, y_train)

# b) report on training and test MSE
y_train_pred = pipe.predict(X_train)
train_mse = mean_squared_error(y_train, y_train_pred)
print(f"Training MSE: {train_mse}")
y_test_pred = pipe.predict(X_test)
test_mse = mean_squared_error(y_test, y_test_pred)
print(f"Test MSE: {test_mse}")


(5329, 11) (1777, 11)
Training MSE: 1.535095636242087
Test MSE: 1.7543308610948491


In [22]:
# Saving my model in joblib

import joblib
joblib.dump(best_model_rs, 'lab-03-final-model.joblib', compress=3)


['lab-03-final-model.joblib']